In [1]:
import pandas as pd
import os

os.chdir("C:/Users/Hegemon/Desktop/DSA 210 Project/Finalized Data")

In [2]:
PrecipDF = pd.read_csv("uncleaned precipitation data era5.csv",header=0)
Temp7WindDF = pd.read_csv("uncleaned temp-wind data era5.csv",header=0)

In [3]:
print(Temp7WindDF)
print(Temp7WindDF["expver"].unique())
print(Temp7WindDF["number"].unique())

         valid_time  latitude  longitude        t2m        d2m       u10  \
0        2000-01-01      48.0     -10.00  283.39453  278.78937  0.450849   
1        2000-01-01      48.0      -9.75  283.39062  278.76398  0.477704   
2        2000-01-01      48.0      -9.50  283.36914  278.73663  0.484540   
3        2000-01-01      48.0      -9.25  283.32617  278.72882  0.477704   
4        2000-01-01      48.0      -9.00  283.31445  278.70538  0.478681   
...             ...       ...        ...        ...        ...       ...   
4646299  2023-12-01      30.0      44.00  287.41504  279.74830 -0.050941   
4646300  2023-12-01      30.0      44.25  287.52440  279.74634  0.179039   
4646301  2023-12-01      30.0      44.50  287.82715  279.52954  0.417320   
4646302  2023-12-01      30.0      44.75  288.02440  279.35962  0.635094   
4646303  2023-12-01      30.0      45.00  288.27050  279.18384  0.811363   

              v10  number  expver  
0       -0.643322       0       1  
1       -0.6237

In [4]:
print(PrecipDF) 
print(PrecipDF["expver"].unique())
print(PrecipDF["number"].unique())

                  valid_time  latitude  longitude        tp  number  expver
0        2000-01-01 06:00:00      48.0     -10.00  0.001701       0       1
1        2000-01-01 06:00:00      48.0      -9.75  0.001774       0       1
2        2000-01-01 06:00:00      48.0      -9.50  0.001775       0       1
3        2000-01-01 06:00:00      48.0      -9.25  0.001729       0       1
4        2000-01-01 06:00:00      48.0      -9.00  0.001624       0       1
...                      ...       ...        ...       ...     ...     ...
4646299  2023-12-01 06:00:00      30.0      44.00  0.000813       0       1
4646300  2023-12-01 06:00:00      30.0      44.25  0.000731       0       1
4646301  2023-12-01 06:00:00      30.0      44.50  0.000631       0       1
4646302  2023-12-01 06:00:00      30.0      44.75  0.000570       0       1
4646303  2023-12-01 06:00:00      30.0      45.00  0.000462       0       1

[4646304 rows x 6 columns]
[1]
[0]


 The columns number and expver are unrelated to the scope of this project since they provide info about the measurement type and scope. In our case they are all the same so it is safe to just remove them.


About the problem of merging the two dataframes, it can be observed that the only difference between them are their valid_time columns, for temp/wind clock is 00:00 and for precipitation clock is 06:00. Since we are not interested with the hour we can just delete them and merge accordingly.

We can also merge wind components into one wind magnitude metric. In addition we can turn K temperature values into Celcius

In [5]:
PrecipDF.drop(columns=["number","expver"],inplace=True)
Temp7WindDF.drop(columns=["number","expver"],inplace=True)

Temp7WindDF["tp"] = PrecipDF["tp"]

merged_weatherDF = Temp7WindDF.copy()

#Reworking the wind speed
merged_weatherDF["wind_speed10"] = (merged_weatherDF["u10"]**2 + merged_weatherDF["v10"]**2)**(0.5)
merged_weatherDF.drop(columns=["u10","v10"],inplace=True)

#Updating Temperatures
merged_weatherDF["t2m"] = merged_weatherDF["t2m"].apply(lambda x: x - 273.15)
merged_weatherDF["d2m"] = merged_weatherDF["d2m"].apply(lambda x: x - 273.15)

print(merged_weatherDF)

         valid_time  latitude  longitude       t2m      d2m        tp  \
0        2000-01-01      48.0     -10.00  10.24453  5.63937  0.001701   
1        2000-01-01      48.0      -9.75  10.24062  5.61398  0.001774   
2        2000-01-01      48.0      -9.50  10.21914  5.58663  0.001775   
3        2000-01-01      48.0      -9.25  10.17617  5.57882  0.001729   
4        2000-01-01      48.0      -9.00  10.16445  5.55538  0.001624   
...             ...       ...        ...       ...      ...       ...   
4646299  2023-12-01      30.0      44.00  14.26504  6.59830  0.000813   
4646300  2023-12-01      30.0      44.25  14.37440  6.59634  0.000731   
4646301  2023-12-01      30.0      44.50  14.67715  6.37954  0.000631   
4646302  2023-12-01      30.0      44.75  14.87440  6.20962  0.000570   
4646303  2023-12-01      30.0      45.00  15.12050  6.03384  0.000462   

         wind_speed10  
0            0.785575  
1            0.785695  
2            0.782947  
3            0.767984  
4  

In [6]:
#Let's check for missing data
print(merged_weatherDF.isna().sum())

valid_time      0
latitude        0
longitude       0
t2m             0
d2m             0
tp              0
wind_speed10    0
dtype: int64


Excellent! There doesn't seem to any missing data, now we have successfully merged precipitation and temp/wind data and checked for missing data.
After this point we need to do two things:
1. Arrange location data into countries
2. Turn monthly data into seasonal/yearly data

In [7]:
import geopandas as gpd

# We remove the duplicate months since ERA5 uses a fixed grid system
unique_coord = merged_weatherDF[["latitude", "longitude"]].drop_duplicates().copy()

unique_gpd = gpd.GeoDataFrame(unique_coord, geometry= gpd.points_from_xy(unique_coord["longitude"], unique_coord["latitude"]), crs="EPSG:4326")
#We instantiate a DataFrame of gpd(it is just a regular DF with a geometry column added,which stores the float location data  as a Point()), 
# GeoDataFrame(
#  data= the same as using pandas,
#  geometry= components of the geometry column are given in gpd.Series,
#  crs= the way to interpret geographic values, for our case we have degrees from Prime Meridian and Equator, thus EPSG:4326 is the code for that)
print(unique_gpd)
#Below one can observe the point data as a column, we will compare the points with country polygons to classify points

       latitude  longitude          geometry
0          48.0     -10.00    POINT (-10 48)
1          48.0      -9.75  POINT (-9.75 48)
2          48.0      -9.50   POINT (-9.5 48)
3          48.0      -9.25  POINT (-9.25 48)
4          48.0      -9.00     POINT (-9 48)
...         ...        ...               ...
16128      30.0      44.00     POINT (44 30)
16129      30.0      44.25  POINT (44.25 30)
16130      30.0      44.50   POINT (44.5 30)
16131      30.0      44.75  POINT (44.75 30)
16132      30.0      45.00     POINT (45 30)

[16133 rows x 3 columns]


In [8]:
url = "https://naturalearth.s3.amazonaws.com/110m_cultural/ne_110m_admin_0_countries.zip"
countries = gpd.read_file(url)[["NAME", "geometry"]]
countries = countries[countries["NAME"].isin(["Turkey", "Spain", "Portugal", "Algeria", "Lebanon","Greece", "Italy", "France", "Cyprus"])]
print(countries)
#This way we were able to filter the polygons of the respective countries to later compare with point location data and classify accordingly

         NAME                                           geometry
43     France  MULTIPOLYGON (((-51.6578 4.15623, -52.24934 3....
77    Lebanon  POLYGON ((35.8211 33.27743, 35.5528 33.26427, ...
82    Algeria  POLYGON ((-8.6844 27.39574, -8.66512 27.58948,...
123    Greece  MULTIPOLYGON (((26.29 35.29999, 26.165 35.005,...
124    Turkey  MULTIPOLYGON (((44.77268 37.17044, 44.29345 37...
131  Portugal  POLYGON ((-9.03482 41.88057, -8.67195 42.13469...
132     Spain  POLYGON ((-7.45373 37.09779, -7.53711 37.4289,...
141     Italy  MULTIPOLYGON (((10.4427 46.89355, 11.04856 46....
161    Cyprus  POLYGON ((32.73178 35.14003, 32.91957 35.08783...


In [9]:
classified_points = gpd.sjoin(unique_gpd, countries, how="left",predicate="within")
# how="left" preserves the main dataframe with point data and adds a resulting column
# predicade= provides the comparison rule, for our case we check if a point is "within" a polygon
print(classified_points)
print(classified_points["NAME"].unique())
# It is expected that points which are not on a country will be classified as NaN so, let's drop them

       latitude  longitude          geometry  index_right NAME
0          48.0     -10.00    POINT (-10 48)          NaN  NaN
1          48.0      -9.75  POINT (-9.75 48)          NaN  NaN
2          48.0      -9.50   POINT (-9.5 48)          NaN  NaN
3          48.0      -9.25  POINT (-9.25 48)          NaN  NaN
4          48.0      -9.00     POINT (-9 48)          NaN  NaN
...         ...        ...               ...          ...  ...
16128      30.0      44.00     POINT (44 30)          NaN  NaN
16129      30.0      44.25  POINT (44.25 30)          NaN  NaN
16130      30.0      44.50   POINT (44.5 30)          NaN  NaN
16131      30.0      44.75  POINT (44.75 30)          NaN  NaN
16132      30.0      45.00     POINT (45 30)          NaN  NaN

[16133 rows x 5 columns]
[nan 'France' 'Italy' 'Spain' 'Portugal' 'Turkey' 'Greece' 'Algeria'
 'Cyprus' 'Lebanon']


In [10]:
classified_points.drop(columns=["index_right"], inplace=True)

merged_weatherDF = merged_weatherDF.merge(classified_points, on=["latitude", "longitude"], how="left")
merged_weatherDF = merged_weatherDF.dropna(axis=0,how="any",ignore_index=True)
print(merged_weatherDF)


         valid_time  latitude  longitude       t2m      d2m        tp  \
0        2000-01-01      48.0      -4.25   7.88906  4.83273  0.001354   
1        2000-01-01      48.0      -4.00   6.67422  4.44600  0.000829   
2        2000-01-01      48.0      -3.75   6.03360  4.01240  0.000743   
3        2000-01-01      48.0      -3.50   5.52773  3.60030  0.000743   
4        2000-01-01      48.0      -3.25   5.41445  3.54952  0.000757   
...             ...       ...        ...       ...      ...       ...   
1463899  2023-12-01      30.0       8.50  12.25918  3.00454  0.000286   
1463900  2023-12-01      30.0       8.75  12.23574  3.23306  0.000389   
1463901  2023-12-01      30.0       9.00  12.17324  3.38345  0.000629   
1463902  2023-12-01      30.0       9.25  12.34120  3.40103  0.000698   
1463903  2023-12-01      30.0       9.50  12.46035  3.49087  0.000738   

         wind_speed10          geometry     NAME  
0            0.520020  POINT (-4.25 48)   France  
1            0.381605

In [11]:
print(merged_weatherDF.isna().sum())
#Now we need to cut some countries(France, Türkiye, Algeria) since we only want the Mediterranean coasts.

FRA_cut = ((merged_weatherDF["NAME"] == "France") & (merged_weatherDF["latitude"] <= 44)) #Bit comparison
TUR_cut = (merged_weatherDF["NAME"] == "Turkey") & (((merged_weatherDF["latitude"] <= 41) & (merged_weatherDF["longitude"] <= 29)) | ((merged_weatherDF["latitude"] <= 38) & (merged_weatherDF["longitude"] <= 41))) #Cutting Aegean and Mediterranean coasts
DZA_cut = ((merged_weatherDF["NAME"] == "Algeria") & (merged_weatherDF["latitude"] >= 35))
ITA_cut = (merged_weatherDF["NAME"] == "Italy")
CYP_cut = (merged_weatherDF["NAME"] == "Cyprus")
LBN_cut = (merged_weatherDF["NAME"] == "Lebanon")
ESP_cut = (merged_weatherDF["NAME"] == "Spain")
PRT_cut = (merged_weatherDF["NAME"] == "Portugal")
GRC_cut = (merged_weatherDF["NAME"] == "Greece")

all_data = merged_weatherDF[DZA_cut|FRA_cut|TUR_cut|ITA_cut|CYP_cut|LBN_cut|ESP_cut|PRT_cut|GRC_cut].copy().reset_index().drop(columns=["index","geometry"])
all_data

valid_time      0
latitude        0
longitude       0
t2m             0
d2m             0
tp              0
wind_speed10    0
geometry        0
NAME            0
dtype: int64


,valid_time,latitude,longitude,t2m,d2m,tp,wind_speed10,NAME
0,2000-01-01,47.00,11.50,-10.45860,-13.93680,0.002026,1.067800,Italy
1,2000-01-01,47.00,11.75,-10.48398,-13.82743,0.002071,0.767301,Italy
2,2000-01-01,47.00,12.00,-10.85312,-14.20830,0.002253,0.651279,Italy
3,2000-01-01,46.75,10.50,-13.18710,-17.52663,0.001801,0.444021,Italy
4,2000-01-01,46.75,10.75,-13.49960,-17.96023,0.001472,1.449759,Italy
...,...,...,...,...,...,...,...,...
791995,2023-12-01,33.75,36.00,9.45254,4.35415,0.002134,0.108119,Lebanon
791996,2023-12-01,33.50,35.50,14.04043,9.45767,0.003918,0.934907,Lebanon
791997,2023-12-01,33.50,35.75,10.42324,6.25454,0.003653,0.387985,Lebanon
791998,2023-12-01,33.25,35.25,17.88418,13.06510,0.007551,1.158103,Lebanon


In [12]:
all_data["valid_time"] = pd.to_datetime(all_data["valid_time"], format="%Y-%m-%d")
all_data.rename(columns={"NAME": "country", "valid_time": "time"},inplace=True)
all_data["year"] = all_data["time"].dt.year
all_data["month"] = all_data["time"].dt.month


yearly_average_by_country_aggregate = all_data.groupby(["country", "year"],as_index=False).agg(temp=("t2m", "mean"), wind_speed10=("wind_speed10", "mean"), total_precip=("tp", "sum"))
seasonal_average_by_country_aggregate = all_data[(all_data["month"] >= 5) & (all_data["month"] <= 9)].groupby(["country", "year"],as_index=False).agg(temp=("t2m", "mean"), wind_speed10=("wind_speed10", "mean"), total_precip=("tp", "sum"))

In [13]:
yearly_average_by_country_aggregate[["year", "country"]] = yearly_average_by_country_aggregate[["country", "year"]]
yearly_average_by_country_aggregate = yearly_average_by_country_aggregate.rename(columns={"country":"year","year":"country"})

seasonal_average_by_country_aggregate[["year", "country"]] = seasonal_average_by_country_aggregate[["country", "year"]]
seasonal_average_by_country_aggregate = seasonal_average_by_country_aggregate.rename(columns={"country":"year","year":"country"})



With this last step the two data frames "yearly_average_by_country_aggregate" and "seasonal_average_by_country_aggregate" have their final usable form

In [14]:
yearly_average_by_country_aggregate

,year,country,temp,wind_speed10,total_precip
0,2000,Algeria,17.139395,1.001420,2.576386
1,2001,Algeria,17.483401,1.049367,3.454161
2,2002,Algeria,17.196522,1.018191,3.621305
3,2003,Algeria,17.306887,0.998623,5.777969
4,2004,Algeria,16.704532,1.005795,5.023409
...,...,...,...,...,...
211,2019,Turkey,15.568469,1.138427,11.829266
212,2020,Turkey,15.775700,1.136894,8.922118
213,2021,Turkey,15.685796,1.188474,9.178751
214,2022,Turkey,15.207696,1.221625,8.745415


In [15]:
seasonal_average_by_country_aggregate

,year,country,temp,wind_speed10,total_precip
0,2000,Algeria,24.445364,0.756066,0.948536
1,2001,Algeria,24.277141,0.807160,0.940279
2,2002,Algeria,23.683132,0.817672,0.969650
3,2003,Algeria,24.915958,0.697662,0.968740
4,2004,Algeria,23.209067,0.783578,1.484135
...,...,...,...,...,...
211,2019,Turkey,23.295509,1.498378,1.631319
212,2020,Turkey,23.619727,1.330505,1.440130
213,2021,Turkey,23.518478,1.623147,1.087326
214,2022,Turkey,23.122259,1.569037,1.676395


In [16]:
yearly_average_by_country_aggregate.to_csv("Final Form Elaborate Yearly Weather.csv", index=False)
seasonal_average_by_country_aggregate.to_csv("Final Form Elaborate Yearly Season Weather.csv", index=False)